In [1]:
%pip install gurobipy -qU

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
items, weight, amount = gp.multidict({
    0: [2,4],
    1: [3,2],
    2: [4,6],
    3: [5,6],
    4: [6,2],
    5: [7,1],
    6: [8,8],
})

capacity = 9
patterns = [[4,0,0,0,0,0,0],[0,3,0,0,0,0,0],[0,0,2,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]]

In [4]:
# Create master model
master = gp.Model("MasterBin")

# Create initial variables. We have 1 initial variable per item.
use = {}
for i in items:
    use[i] = master.addVar()

# Create master objective
master.setObjective(sum(use[i] for i in items), GRB.MINIMIZE)

# Constraints
demand_cons = master.addConstrs(gp.quicksum(patterns[j][i]*use[i] for j in range(0,len(patterns))) >= amount[i] for i in items)

Restricted license - for non-production use only - expires 2025-11-24


In [5]:
# Create pricing model
pricing = gp.Model("PricingBin")

# Create pricing variables
select = pricing.addVars(items, name="select", vtype=GRB.INTEGER)

# Create constraint
pricing.addConstr(gp.quicksum(select[i]*weight[i] for i in items) <= capacity)


<gurobi.Constr *Awaiting Model Update*>

In [6]:

while 1:
    master.optimize()
    pricing.setObjective(gp.quicksum(demand_cons[i].Pi*select[i] for i in items), GRB.MAXIMIZE)
    pricing.optimize()
    # If a new pattern is found, we add it to the master problem.
    if pricing.ObjVal > 1:
        improve = 1
        new_pattern = []
        for i in items:
            new_pattern.append(select[i].X)
        # Due to numerical issues, it is possible that patterns incorrectly get identified as improving. If this happens, the same
        # patterns will keep being added, and not being used in the master problem. The following statement is to identify this
        # situation if it happens.
        if new_pattern == patterns[len(patterns)-1]:
            break
        patterns.append(new_pattern)
        # Build the new variable (and include in the objective with coefficient 1)
        use[len(patterns)-1] = master.addVar(obj = 1)
        # This includes the new variable/pattern in the constraints.
        for i in items:
            master.chgCoeff(demand_cons[i], use[len(patterns)-1], new_pattern[i])
    else:
        break
        

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 7 columns and 7 nonzeros
Model fingerprint: 0x06127c8d
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 7 rows and 7 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1666667e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.166666667e+01
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical 

In [7]:
def printSolution():
    if master.status == GRB.OPTIMAL:
        print('\nCost: %g' % master.ObjVal)
        print('\nUse:')
        for i in range(0,len(patterns)):
            if use[i].X > 0.0001:
                print('%s %g' % (patterns[i], use[i].X))
    else:
        print('No solution')
        
printSolution()


Cost: 17.75

Use:
[4, 0, 0, 0, 0, 0, 0] 0.75
[0, 0, 0, 0, 0, 0, 1] 8
[-0.0, 0.0, 1.0, 1.0, -0.0, -0.0, -0.0] 6
[0.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0] 2
[1.0, 0.0, -0.0, -0.0, -0.0, 1.0, -0.0] 1


In [8]:
master.write("binMaster.lp")
f = open("binMaster.lp","r")
contents = f.read()
print(contents)

\ Model MasterBin
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  C0 + C1 + C2 + C3 + C4 + C5 + C6 + C7 + C8 + C9 + C10 + C11
Subject To
 R0: 4 C0 + 2 C7 + C10 + 3 C11 >= 4
 R1: 3 C1 + C9 + C11 >= 2
 R2: 2 C2 + C8 >= 6
 R3: C3 + C7 + C8 >= 6
 R4: C4 + C9 >= 2
 R5: C5 + C10 >= 1
 R6: C6 >= 8
Bounds
End



In [9]:
pricing.write("binPrice.lp")
f = open("binPrice.lp","r")
contents = f.read()
print(contents)

\ Model PricingBin
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  0.25 select[0] + 0.25 select[1] + 0.5 select[2] + 0.5 select[3]
   + 0.75 select[4] + 0.75 select[5] + select[6]
Subject To
 R0: 2 select[0] + 3 select[1] + 4 select[2] + 5 select[3] + 6 select[4]
   + 7 select[5] + 8 select[6] <= 9
Bounds
Generals
 select[0] select[1] select[2] select[3] select[4] select[5] select[6]
End

